In [2]:
# Import libraries

!pip install pretty_midi
!pip install tensorflow
!pip install numpy
!pip install pandas
!pip install tensorflow.keras



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 7.2 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592284 sha256=9f77848820b1e7056ad9c5d3eb7b3035775ac09e7c4170d8f9eff09a882d3b83
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
ERROR: Could not find a version that satisfies the requirement tensorflow.keras (from versions: none)
ERROR: No matching distribution found for tensorflow.keras


In [8]:
import os
import zipfile

# Specify the zip file path
zip_file_path = "/content/archive (1).zip"

# Extract all files to a temporary directory
with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall(path="extracted_maestro")

# Iterate through extracted folders and collect MIDI files
midi_files = []
for root, _, files in os.walk("extracted_maestro"):
    for filename in files:
        if filename.endswith(".mid"):
            midi_path = os.path.join(root, filename)
            midi_files.append(midi_path)

print(f"Found {len(midi_files)} MIDI files:")

# Print extracted MIDI files (optional)
for file in midi_files:
    print(file)

with open("extracted_midi_files.txt", "a") as file:
    for midi_file in midi_files:
        file.write(midi_file + "\n")

print(f"Successfully appended {len(midi_files)} MIDI files to extracted_midi_files.txt")




Found 93 MIDI files:
extracted_maestro/Cymatics - Lofi MIDI 2 - C Maj.mid
extracted_maestro/Piano Chords MIDI (2).mid
extracted_maestro/Cymatics - Eternity MIDI 12 - A Min.mid
extracted_maestro/9.mid
extracted_maestro/Rhodes MIDI (6).mid
extracted_maestro/Cymatics - Lofi MIDI 19 - G Maj.mid
extracted_maestro/Cymatics - Lofi MIDI 9 - D Maj.mid
extracted_maestro/Cymatics - Lofi MIDI 16 - F Maj.mid
extracted_maestro/8.mid
extracted_maestro/Cymatics - Eternity MIDI 11 - A Maj.mid
extracted_maestro/20.mid
extracted_maestro/Cymatics - Lofi MIDI 1 - C Maj.mid
extracted_maestro/Cymatics - Eternity MIDI 3 - D Maj.mid
extracted_maestro/Cymatics - Eternity MIDI 6 - D Min.mid
extracted_maestro/Cymatics - Lofi MIDI 20 - G Min.mid
extracted_maestro/7.mid
extracted_maestro/Piano MIDI.mid
extracted_maestro/17.mid
extracted_maestro/Piano MIDI (7).mid
extracted_maestro/Cymatics - Eternity MIDI 8 - F Maj.mid
extracted_maestro/Cymatics - Eternity MIDI 13 - A Min.mid
extracted_maestro/Cymatics - Lofi MIDI 

In [9]:
# Define an empty list to store MIDI file paths
songs = []

# Open the file containing extracted MIDI file paths
with open("/content/extracted_midi_files.txt", "r") as file:
    # Read each line as a MIDI file path
    for line in file:
        midi_path = line.strip()  # Remove trailing whitespace
        songs.append(midi_path)

print(f"Successfully read {len(songs)} MIDI file paths from /content/extracted_midi_files.txt")



Successfully read 279 MIDI file paths from /content/extracted_midi_files.txt


In [11]:
import random

# Get a subset of 250 songs
result = random.sample(songs, 250)

#print(f"Sampled {len(result)} MIDI file paths from the list.")



In [12]:
from music21 import converter, instrument, note, chord
notes = []
for i,file in enumerate(result):
    print(f'{i+1}: {file}')
    try:
      midi = converter.parse(file)
      notes_to_parse = None
      parts = instrument.partitionByInstrument(midi)
      if parts: # file has instrument parts
          notes_to_parse = parts.parts[0].recurse()
      else: # file has notes in a flat structure
          notes_to_parse = midi.flat.notes
      for element in notes_to_parse:
          if isinstance(element, note.Note):
              notes.append(str(element.pitch))
          elif isinstance(element, chord.Chord):
              notes.append('.'.join(str(n) for n in element.normalOrder))
    except:
      print(f'FAILED: {i+1}: {file}')



1: extracted_maestro/Cymatics - Eternity MIDI 8 - F Maj.mid
2: extracted_maestro/Cymatics - Eternity MIDI 20 - A Min.mid
3: extracted_maestro/Piano MIDI 2.mid
4: extracted_maestro/Cymatics - Eternity MIDI 15 - A Maj.mid
5: extracted_maestro/Piano Chords MIDI (3).mid
6: extracted_maestro/10.mid
7: extracted_maestro/Cymatics - Lofi MIDI 16 - F Maj.mid
8: extracted_maestro/15.mid
9: extracted_maestro/12.mid
10: extracted_maestro/Cymatics - Eternity MIDI 8 - F Maj.mid
11: extracted_maestro/Piano 1 MIDI.mid
12: extracted_maestro/Cymatics - Eternity MIDI 13 - A Min.mid
13: extracted_maestro/Piano MIDI (5).mid
14: extracted_maestro/Cymatics - Lofi MIDI 17 - G Maj.mid
15: extracted_maestro/Cymatics - Eternity MIDI 13 - A Min.mid
16: extracted_maestro/Rhodes MIDI (3).mid
17: extracted_maestro/Piano MIDI (7).mid
18: extracted_maestro/Rhodes MIDI (9).mid
19: extracted_maestro/Cymatics - Lofi MIDI 5 - D Maj.mid
20: extracted_maestro/Cymatics - Eternity MIDI 17 - A Maj.mid
21: extracted_maestro/Cym

In [13]:

import pickle
with open('notes', 'wb') as filepath:
  pickle.dump(notes, filepath)



In [14]:
import numpy as np

In [15]:
from keras.utils import to_categorical  # Import to_categorical function from Keras utilities

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 32

    # Get all unique pitchnames
    pitchnames = sorted(set(item for item in notes))
    numPitches = len(pitchnames)

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)

    # One-hot encode the network_output
    network_output = to_categorical(network_output, num_classes=numPitches)  # One-hot encoding

    return network_input, network_output


n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)
n_patterns = len(network_input)
pitchnames = sorted(set(item for item in notes))
numPitches = len(pitchnames)


In [16]:
import tensorflow as tf
print(tf.__version__)



2.15.0


In [17]:
!pip install keras



In [18]:
pip install np_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 900.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56439 sha256=a044965a5e4b1413c78109098e6f57ac455bb7d6cc0eecab13fae867bfc077f3
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils


In [19]:
import pandas as pd

In [20]:
def oversample(network_input, network_output, sequence_length=15):
    """ Oversample the dataset """
    n_patterns = len(network_input)

    # Create a DataFrame from the two matrices
    new_df = pd.concat([pd.DataFrame(network_input), pd.DataFrame(network_output)], axis=1)

    # Rename the columns to numbers and Notes
    new_df.columns = [x for x in range(sequence_length + 1)]
    new_df = new_df.rename(columns={sequence_length: 'Notes'})

    # Print information about the pre-oversampled DataFrame
    print(new_df.tail(20))
    print('###################################################')
    print(f'Distribution of notes in the preoversampled DataFrame: {new_df["Notes"].value_counts()}')

    # Oversampling
    oversampled_df = new_df.copy()
    max_class_size = np.max(oversampled_df['Notes'].value_counts())
    print('Size of biggest class: ', max_class_size)

    class_subsets = [oversampled_df.query('Notes == ' + str(i)) for i in range(len(new_df["Notes"].unique()))]

    # Oversample each class subset
    for i in range(len(new_df['Notes'].unique())):
        try:
            class_subsets[i] = class_subsets[i].sample(max_class_size, random_state=42, replace=True)
        except:
            print(i)

    # Concatenate the oversampled class subsets and shuffle the data
    oversampled_df = pd.concat(class_subsets, axis=0).sample(frac=1.0, random_state=42).reset_index(drop=True)

    # Print information about the oversampled DataFrame
    print('###################################################')
    print(f'Distribution of notes in the oversampled DataFrame: {oversampled_df["Notes"].value_counts()}')

    # Sample from the oversampled DataFrame
    sampled_df = oversampled_df.sample(n_patterns, replace=True)

    # Update networkInputShaped with the sampled data shape
    networkInputShaped = sampled_df.shape

    # Print information about the sampled DataFrame
    print('###################################################')
    print(f'Distribution of notes in the oversampled post-sampled DataFrame: {sampled_df["Notes"].value_counts()}')

    # Convert the training columns to a 3D array
    network_in = sampled_df[[x for x in range(sequence_length)]]
    network_in = np.reshape(network_in, (n_patterns, sequence_length, 1)) / numPitches

    # Convert the target column to one-hot encoding
    network_out = pd.get_dummies(oversampled_df['Notes'])

    return network_in, network_out



In [21]:
print(network_input.shape)

print(network_output.shape)




(5023, 32, 1)
(5023, 254)


In [22]:
network_input_reshaped = network_input.reshape((network_input.shape[0], network_input.shape[1]))
print(f"Shape of network_input_reshaped: {network_input_reshaped.shape}")
network_output_reshaped = network_output.reshape((network_output.shape[0], network_output.shape[1]))
print(f"Shape of network_input_reshaped: {network_output_reshaped.shape}")

Shape of network_input_reshaped: (5023, 32)
Shape of network_input_reshaped: (5023, 254)


In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout


In [24]:
# Define the LSTM model architecture
model = Sequential()

# Dropout layer to avoid overfitting
model.add(Dropout(0.2))

# First LSTM layer with 512 units, accepting input sequences and returning sequences
model.add(LSTM(512, return_sequences=True, input_shape=(277384, 32)))




# Dense hidden layer with 256 units
model.add(Dense(256))

# Second LSTM layer with 256 units, accepting sequences and returning sequences
model.add(LSTM(256, return_sequences=True))

# Dense hidden layer with 256 units
model.add(Dense(256))

# Final LSTM layer with 512 units
model.add(LSTM(512))

# Output layer with the same number of units as the number of unique pitches
model.add(Dense(numPitches))

# Apply softmax activation function for multi-class classification
model.add(Activation('softmax'))

# Compile the model with categorical cross-entropy loss, RMSProp optimizer, and accuracy metric
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])



In [25]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the number of epochs and callback list
num_epochs = 100

# Define the filepath for saving model checkpoints
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger_1.hdf5"

# Create a ModelCheckpoint callback to save the best model during training
checkpoint = ModelCheckpoint(
    filepath=filepath,
    monitor="loss",
    verbose=1,
    save_best_only=True,
    mode="min",
)

# Create a list of callbacks to be used during training
callbacks_list = [checkpoint]

# Train the LSTM model
history = model.fit(
    network_input,
    network_output,
    epochs=num_epochs,
    batch_size=64,
    callbacks=callbacks_list,
)



Epoch 1/100
79/79 [==============================] - ETA: 0s - loss: 5.0502 - accuracy: 0.0183
Epoch 1: loss improved from inf to 5.05020, saving model to weights-improvement-01-5.0502-bigger_1.hdf5
79/79 [==============================] - 17s 23ms/step - loss: 5.0502 - accuracy: 0.0183
Epoch 2/100
 4/79 [>.............................] - ETA: 1s - loss: 4.8463 - accuracy: 0.0391

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


79/79 [==============================] - ETA: 0s - loss: 4.9620 - accuracy: 0.0257
Epoch 2: loss improved from 5.05020 to 4.96204, saving model to weights-improvement-02-4.9620-bigger_1.hdf5
79/79 [==============================] - 2s 23ms/step - loss: 4.9620 - accuracy: 0.0257
Epoch 3/100
79/79 [==============================] - ETA: 0s - loss: 4.9431 - accuracy: 0.0229
Epoch 3: loss improved from 4.96204 to 4.94307, saving model to weights-improvement-03-4.9431-bigger_1.hdf5
79/79 [==============================] - 2s 23ms/step - loss: 4.9431 - accuracy: 0.0229
Epoch 4/100
79/79 [==============================] - ETA: 0s - loss: 4.9091 - accuracy: 0.0203
Epoch 4: loss improved from 4.94307 to 4.90912, saving model to weights-improvement-04-4.9091-bigger_1.hdf5
79/79 [==============================] - 2s 22ms/step - loss: 4.9091 - accuracy: 0.0203
Epoch 5/100
79/79 [==============================] - ETA: 0s - loss: 4.8850 - accuracy: 0.0255
Epoch 5: loss improved from 4.90912 to 4.884

In [26]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    # Pick a random sequence from the input as a starting point
    start = np.random.randint(0, len(network_input) - 1)
    # Random row from network_input
    pattern = network_input[start]
    prediction_output = []

    # Generate 500 notes
    for note_index in range(500):
        # Reshapes pattern into a vector
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        # Standardizes pattern
        prediction_input = prediction_input / float(n_vocab)

        # Predicts the next note
        prediction = model.predict(prediction_input)

        # Outputs a OneHot encoded vector, so this picks the column
        # with the highest probability
        index = np.argmax(prediction)
        # Maps the note to its respective index
        result = pitchnames[index]

        # Appends the note to the prediction_output
        prediction_output.append(result)

        # Adds the predicted note to the pattern
        pattern = np.append(pattern, index)
        # Slices the array so that it contains the predicted note
        # eliminating the first from the array, so the model can
        # have a sequence
        pattern = pattern[1:len(pattern)]

    return prediction_output


In [27]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    start = np.random.randint(0, len(network_input) - 1)
    pattern = network_input[start]
    prediction_output = []

    print(f"Initial Pattern: {pattern}")  # Check the initial pattern

    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        print(f"Prediction Input Shape: {prediction_input.shape}")  # Check shapes

        prediction = model.predict(prediction_input)

        print(f"Prediction Shape: {prediction.shape}")  # Check prediction shape
        print(f"Prediction Values: {prediction}")  # Check prediction values

        index = np.argmax(prediction)
        result = pitchnames[index]

        print(f"Selected Index: {index}")  # Check the selected index
        print(f"Selected Note: {result}")  # Check the selected note

        prediction_output.append(result)

        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]

    return prediction_output


In [28]:
# Call the function with prepared data
prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)

# Print the generated notes
print(prediction_output)


Streaming output truncated to the last 5000 lines.
Selected Note: B-4
Prediction Input Shape: (1, 32, 1)
1/1 [==============================] - 0s 41ms/step
Prediction Shape: (1, 254)
Prediction Values: [[4.35528578e-04 1.16392551e-03 6.40785656e-05 1.64115697e-03
  8.09272286e-03 3.13600030e-04 1.92221138e-04 1.90857367e-03
  4.70839179e-04 1.23575539e-03 1.35467120e-03 4.25865815e-04
  2.18365225e-03 1.90339214e-03 7.11360434e-03 6.50598900e-03
  1.82851858e-03 5.49487071e-04 3.85942520e-03 3.56530247e-04
  5.89658041e-04 6.12453034e-04 6.83385122e-04 3.40637751e-04
  6.14312070e-04 8.59315740e-04 2.65875040e-03 8.89156852e-03
  1.23171613e-03 7.72561121e-04 1.67479520e-04 6.98291638e-04
  1.11223012e-03 4.81420750e-04 4.27024061e-04 2.01807730e-03
  4.89451550e-02 3.62978596e-03 7.19368923e-04 2.17526453e-03
  6.87865715e-04 1.38930758e-04 5.12798215e-05 1.07573622e-04
  7.70050217e-04 6.45530992e-04 1.89304829e-03 4.46741731e-04
  5.39214525e-04 5.92542579e-04 3.20947875e-04 1.7828

In [29]:
from music21 import note, chord, instrument, stream

def create_midi(prediction_output):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            # Processing a chord
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            # Processing a single note
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='output.mid')
    return midi_stream



# Create the MIDI file
generated_midi = create_midi(prediction_output)

# Print information about the prediction output and the generated MIDI stream
print(f"Number of elements in prediction output: {len(prediction_output)}")
print(f"First element in prediction output: {prediction_output[0]}")
print(f"MIDI stream: {generated_midi}")



Number of elements in prediction output: 500
First element in prediction output: B-5
MIDI stream: <music21.stream.Stream 0x7edda7a5cd90>


In [30]:
from pathlib import Path

# Define the folder path where the HDF5 files are located
folder = Path('/content')

# Collect the paths of the HDF5 files
songs = [str(file) for file in folder.rglob('*.hdf5')]  # List comprehension for paths

# Define the file where you want to save the collected paths
output_file = 'all_songs.txt'  # Change this to your desired file name

# Write the collected paths into the output file
with open(output_file, 'w') as f:
    f.write('\n'.join(songs))  # Write all paths joined by newlines

print(f"Collected {len(songs)} songs. Saved paths in {output_file}.")






Collected 90 songs. Saved paths in all_songs.txt.


In [ ]:

songsList = []
weightsList = []
for i in range(len(songs)):
  try:
    model.load_weights(songs[i])
    prediction_output = generate_notes(model, network_input_reshaped, pitchnames, n_vocab)
    songsList.append(prediction_output)
    weightsList.append(str(songs[i]))
  except:
    pass

songs_df = pd.DataFrame({'Weights':weightsList,
                         'Notes':songsList})


Streaming output truncated to the last 5000 lines.
  1.30411907e-04 1.76060712e-04 2.99852254e-04 7.05656275e-05
  2.59080873e-04 1.02581899e-03 5.71087403e-05 1.14493334e-04
  1.13687100e-04 2.40425419e-04 3.06456932e-03 5.87773829e-05
  1.28452573e-03 1.69598730e-03 1.45612968e-04 1.45400802e-04
  5.23729250e-04 5.57725725e-04 1.39458897e-03 3.44002561e-04
  1.03963283e-03 4.11857363e-05 1.52068635e-04 3.55726341e-04
  2.95285718e-03 9.21551255e-04 2.27660930e-04 1.62008117e-04
  6.73492177e-05 2.84813490e-04 2.65453564e-04 9.92327812e-04
  6.23613669e-05 1.18768905e-04 1.87091166e-04 3.45433108e-03
  7.62986543e-04 2.94591263e-02 5.50455588e-04 6.11677649e-04
  1.43968835e-04 4.69914032e-03 1.11258872e-04 1.74759573e-03
  8.39848537e-03 2.68950062e-05 1.70059269e-03 2.84041074e-04
  4.11037254e-05 4.70454775e-04 1.88030954e-03 4.46836551e-04
  1.49523036e-03 7.65911027e-05 4.21455828e-04 1.47369399e-03
  1.13080651e-03 4.28518797e-05 2.22431947e-04 2.97595019e-04
  4.08265878e-05 2.

In [31]:
pip install pygame

In [33]:
from music21 import converter, midi

def play_midi(midi_file_path):
    mf = midi.MidiFile()
    mf.open(midi_file_path)
    mf.read()
    mf.close()

    s = midi.translate.midiFileToStream(mf)
    s.show('midi')

# Replace 'path/to/your/output.mid' with your MIDI file path
midi_file_path = '/content/output.mid'

# Call the function with your MIDI file path
play_midi(midi_file_path)

